### Import necessary packages

In [1]:
!pip install transformers seqeval[gpu]
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 6.6 MB 57.5 MB/s 
     |████████████████████████████████| 596 kB 85.2 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=fcd7fe62f1d85d73d443398639eed374e81567f3124250468b2a0ebf1ec49b5b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.1 MB/s 
     |█████████████████

In [2]:
import torch
import numpy as np
import pandas as pd
import datasets
from collections import Counter
from datasets import load_dataset
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1, 1, 1, 1, 1, 1, 0.38, 1, 0.66, 1, 1]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

### Load data

In [5]:
torch.cuda.is_available()
device = torch.device("cuda")
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/eth_CS4NLP/project/qadata'
train_path = '/content/drive/MyDrive/eth_CS4NLP/project/qadata/df_traindev.csv'
dev_path = '/content/drive/MyDrive/eth_CS4NLP/project/qadata/final_dev.csv'
test_path = '/content/drive/MyDrive/eth_CS4NLP/project/qadata/vardial_test.txt'

Mounted at /content/drive
df_traindev.csv  pred_bert_output	vardial_train_1.csv
final_dev.csv	 Record_outputs.gsheet	vardial_train_2.csv
final_train.csv  vardial_dev.txt
output		 vardial_test.txt


In [6]:
dial_label = {
    0 : 'EML',
    1 : 'NAP',
    2 : 'PMS',
    3 : 'FUR',
    4 : 'LLD',
    5 : 'LIJ',
    6 : 'LMO',
    7 : 'ROA_TARA',
    8 : 'SCN', 
    9 : 'VEC',
    10 : 'SC'
}

fold_label = {
    'EML' : 0,
    'NAP' : 1,
    'PMS' : 2,
    'FUR' : 3,
    'LLD' : 4,
    'LIJ' : 5,
    'LMO' : 6,
    'ROA_TARA' : 7,
    'SCN' : 8,
    'VEC' : 9,
    'SC' : 10
}

def explain_label(label : int) -> str:
    """ 
    Given an integer label, convert it to the corresponding string label
    :param int label: integer label to be converted
    :return: string corresponding to the given label
    """
    return dial_label[label]

def encode_label(label : str) -> int:
    """ 
    Given a string label, encode it to the corresponding index
    :param string label: string label to be converted
    :return: int corresponding to the given label
    """
    return fold_label[label]

In [7]:
df_dev = pd.read_csv(dev_path)
df_train = pd.read_csv(train_path)
# testset
df_test = pd.read_csv(test_path, sep = "\t", names=["text"])
df_test['label']=0 #add label column to the testset and assign 0 to all row
df_test.shape

(11087, 2)

In [8]:
# Create dictionaries to transform from labels to id and vice-versa.
id2label = {0 : 'EML',
    1 : 'NAP',
    2 : 'PMS',
    3 : 'FUR',
    4 : 'LLD',
    5 : 'LIJ',
    6 : 'LMO',
    7 : 'ROA_TARA',
    8 : 'SCN', 
    9 : 'VEC',
    10 : 'SC'}
label2id = {v:k for k,v in id2label.items()}
num_labels = len(id2label)
print(id2label)

{0: 'EML', 1: 'NAP', 2: 'PMS', 3: 'FUR', 4: 'LLD', 5: 'LIJ', 6: 'LMO', 7: 'ROA_TARA', 8: 'SCN', 9: 'VEC', 10: 'SC'}


In [9]:
from datasets import Dataset
train_ds = Dataset.from_pandas(df_train)
dev_ds = Dataset.from_pandas(df_dev)
test_ds = Dataset.from_pandas(df_test)
# Example of instance of the dataset
train_ds[18]

{'id': 32711.0,
 'label': 0,
 'text': " 'l è 'l nòm 'd un domìni genèric. Al funsiòuna da 'l dicèmber dal 2016.",
 'title': '.كاثوليك',
 'url': 'https://eml.wikipedia.org/wiki?curid=32711'}

## Tokenizing data - Model name

**model_name**
- [] m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0
- [] 'dbmdz/bert-base-italian-cased'
- [x] 'dbmdz/bert-base-italian-xxl-cased'
- [] 'dbmdz/bert-base-italian-uncased'
- [] mrm8488/bert-italian-finedtuned-squadv1-it-alfa

In [11]:

# Load BERT tokenizer.
model_name = 'dbmdz/bert-base-italian-xxl-cased'


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

In [13]:

def encode_dataset(dataset: datasets.arrow_dataset.Dataset) -> list:
  '''
  Transforming each instance of the dataset with the Tokenizer
  '''
  encoded_dataset = []
  for item in dataset:
    # Tokenize the sentence.
    sentence_encoded = tokenizer(item['text'],
                                return_tensors="pt", 
                                padding='max_length', 
                                truncation=True, 
                                max_length=50)
    
    sentence_encoded['labels'] = torch.LongTensor(np.array([item['label']]))
    encoded_dataset.append(sentence_encoded)

  # Reduce dimensionality of tensors.
  for item in encoded_dataset:
    for key in item:
      item[key] = torch.squeeze(item[key])
  return encoded_dataset

In [14]:
# Tokenizing datasets
encoded_dataset_train = encode_dataset(train_ds)
encoded_dataset_dev = encode_dataset(dev_ds)
encoded_dataset_test = encode_dataset(test_ds)


In [15]:
# Example of dataset
for token, token_label in zip(tokenizer.convert_ids_to_tokens(encoded_dataset_train[50]["input_ids"]), encoded_dataset_train[50]["input_ids"]):
  print('{0:10}  {1}'.format(token, token_label))

[CLS]       102
I           184
an          221
1970        12365
,           1307
cia         2718
##m         30889
##ê         31028
an          221
Est         11965
##à         30914
##n         30880
##ta        115
,           1307
i           134
[UNK]       101
chi         524
[UNK]       101
an          221
ch          471
'           1553
i           134
partì       22841
##sen       6775
da          203
'           1553
l           181
1970        12365
inf         1238
##ì         30946
##n         30880
a           111
'           1553
l           181
1979        10542
cia         2718
##p         30888
##ê         31028
dèi         28237
##nte       1696
##r         30882
.           697
[SEP]       103
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0


# Fine tunning of the model

In [16]:
# Common training arguments
training_args = TrainingArguments(
    num_train_epochs=2,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_dir='logs',
    no_cuda=False,  
    output_dir = 'drive/MyDrive/eth_CS4NLP/project',
)

# Dictionary to save the results
models_performance ={}

# FINE TUNING PROCESS

# create model
#model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=num_labels)#,class_weights=[1, 1, 1, 1, 1, 1, 0.38, 1, 0.66, 1, 1] )
#0:1, 1:1, 2:1, 3:1, 4:1, 5:1, 6:0.38, 7:1, 8:0.66, 9:1, 10:1

'''model.transformer.mask_emb.requires_grad = False
model.transformer.word_embedding.weight.requires_grad = False
for name, param in model.transformer.layer.named_parameters():
  try:
    layer = int(name[:2])
  except ValueError:
    try:
      layer = int(name[:1])
    except ValueError:
      layer = 0
  if layer <= 20:
    param.requires_grad = False'''


# Create trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=encoded_dataset_train,)

# Fine tunning
trainer.train()


Downloading:   0%|          | 0.00/425M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

Step,Training Loss
500,0.133600
1000,0.027700
1500,0.023200
2000,0.016700
2500,0.008200
3000,0.009300
3500,0.006700


Saving model checkpoint to drive/MyDrive/eth_CS4NLP/project/checkpoint-500
Configuration saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-500/config.json
Model weights saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-500/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-500/tokenizer_config.json
Special tokens file saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-500/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/eth_CS4NLP/project/checkpoint-1000
Configuration saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-1000/tokenizer_config.json
Special tokens file saved in drive/MyDrive/eth_CS4NLP/project/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/eth_CS4NLP/project/checkpoint-1500
Configuration saved i

TrainOutput(global_step=3934, training_loss=0.029329269988075026, metrics={'train_runtime': 2170.4675, 'train_samples_per_second': 231.988, 'train_steps_per_second': 1.813, 'total_flos': 1.29387607768866e+16, 'train_loss': 0.029329269988075026, 'epoch': 2.0})

# Models evaluation

In [ ]:
# Get the predicted labels
preds = trainer.predict(encoded_dataset_dev)
predictions = preds.predictions.argmax(-1)

# Create array with predicted labels and expected.
true_values = np.array(preds.label_ids).flatten()
predicted_values = np.array(preds.predictions.argmax(-1)).flatten()

# Filter the labels. We only produce a label for each word. We filter labels
# of subwords and special tokens, such as PAD
proc_predicted_values = [prediction for prediction, label in zip(predicted_values, true_values) if label != -100]
proc_true_values = [label for prediction, label in zip(predicted_values, true_values) if label != -100]

# Evaluate models
model_performance = {}
model_performance['accuracy'] = accuracy_score(proc_true_values, proc_predicted_values)
model_performance['precision_micro'] = precision_score(proc_true_values, proc_predicted_values, average='micro')
model_performance['precision_macro'] = precision_score(proc_true_values, proc_predicted_values, average='macro')
model_performance['recall_micro'] = recall_score(proc_true_values, proc_predicted_values, average='micro')
model_performance['recall_macro'] = recall_score(proc_true_values, proc_predicted_values, average='macro')
model_performance['f1_micro'] = f1_score(proc_true_values, proc_predicted_values, average='micro')
model_performance['f1_macro'] = f1_score(proc_true_values, proc_predicted_values, average='macro')

model_performance['confusion_matrix'] = confusion_matrix(proc_true_values, proc_predicted_values)
model_performance['confusion_matrix_normalized'] = confusion_matrix(proc_true_values, proc_predicted_values, normalize='true')

In [ ]:
model_performance

F1-micro and F1-macro for each model

In [ ]:
print('------------Model performance------------')
print(f'  accuracy: {model_performance["accuracy"]}')
print(f'  f1-micro: {model_performance["f1_micro"]}')
print(f'  f1-macro: {model_performance["f1_macro"]}')
print(f'  precision_macro: {model_performance["precision_macro"]}')
print(f'  precision_micro: {model_performance["precision_micro"]}')
print(f'  recall_macro: {model_performance["recall_macro"]}')
print(f'  recall_micro: {model_performance["recall_micro"]}')
print()

In [ ]:
pred_df = pd.DataFrame({'pred_label':predicted_values})
pred_df

In [ ]:
from google.colab import files
pred_df.to_csv('new_pred_dbmdz_bert-base-italian-xxl-cased.csv',index=False)
files.download('new_pred_dbmdz_bert-base-italian-xxl-cased.csv')

# Model Prediction

In [17]:
# Get the predicted labels
preds = trainer.predict(encoded_dataset_test)
predictions = preds.predictions.argmax(-1)

# Create array with predicted labels and expected.
true_values = np.array(preds.label_ids).flatten()
predicted_values = np.array(preds.predictions.argmax(-1)).flatten()

***** Running Prediction *****
  Num examples = 11087
  Batch size = 128


In [19]:
pred_test = pd.DataFrame({'pred_label':predicted_values})
from google.colab import files
pred_test.to_csv('finalbert_test_pred.csv',index=False)
files.download('finalbert_test_pred.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [61]:
from numpy import savetxt
# save to csv file
savetxt('logit.csv', logits, delimiter=',')
files.download('logit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [73]:
#softmax on the logits
qa = preds[0]
for i in range (3):
  input_array = qa[i]
  input = torch.from_numpy(input_array)
  m = nn.Softmax()
  qa[i] = m(input)
  from numpy import savetxt
savetxt('logit_softmax.csv', qa, delimiter=',')
files.download('logit_softmax.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [42]:
arrayA = preds[0]
arrayB = predictions
df_preds = pd.DataFrame(zip(arrayA, arrayB), columns=['logits','predicted_label'])
df_preds.head(3)
'''df_preds.to_csv('finalbert_probability.csv',index=False)
files.download('finalbert_probability.csv')'''

"df_preds.to_csv('finalbert_probability.csv',index=False)\nfiles.download('finalbert_probability.csv')"

In [63]:
 
qa = pd.read_csv('/content/logit.csv')
qa.head()

,-2.127698659896850586e+00,-4.896732568740844727e-01,-2.544776678085327148e+00,2.180627137422561646e-01,-3.567491054534912109e+00,3.186703622341156006e-01,-8.513054251670837402e-01,-1.167246818542480469e+00,-2.384972572326660156e-01,6.014449596405029297e+00,1.599810719490051270e+00
0,4.734923,-2.860902,0.278604,-1.951446,-0.426649,-1.604621,1.941120,-2.621810,-1.789820,4.066730,-1.950771
1,-0.503288,-1.257475,-0.957177,9.129163,-1.378020,-0.924327,-0.794488,-0.561070,-0.931509,-1.323981,-0.393470
2,-1.603751,0.753515,0.793998,-0.730342,-2.030601,-1.567282,-1.824576,-0.427938,4.719806,2.369624,-0.171318
3,-0.798203,-1.060067,-0.682767,9.140423,-0.632034,-1.430424,-0.877635,-0.686138,-0.709928,-1.336049,-0.599793
4,-1.266560,5.491029,-1.821992,-0.333703,-1.902554,-1.224146,-0.800067,5.875556,-0.778336,-0.495845,-1.032106
